# Lecture 5

In [3]:
import numpy as np

### Evaluating $e^{-x}$

When we Taylor expand $e^{-x}$, we have $1 - \frac{x}{1!} + \frac{x^{2}}{2!} - \frac{x^{3}}{3!} + ... + \frac{x^{n}}{n!}$. We can apply this to our programme. One method is to isolate odd and even terms. Let us initialise a variables `sum`, `sumeven` and `sumodd`.

1. Loop over `k` from 1 to $n$
2. `termeven` = `termeven` $\times \frac{x^{2}}{2k(k - 1)}$ and `sumeven = sumeven + termeven`. So when `k = 0`, `termeven = 1` and we have the rest of the odd terms
3. Now, we can evaluate the odd terms: `termodd` = `termodd` $\times \frac{x^{2}}{(2k + 1)2k}$. So, `sumodd = sumodd - termodd`.

Essentially we have:

`sum` $= \sum_{k = 0}^{n} \frac{x^{k}}{k!}$

`sumeven` $= \sum_{k = 0}^{n/2} \frac{x^{2k}}{(2k)!}$

`sumodd` $= \sum_{k = 0}^{n/2} \frac{x^{2k + 1}}{(2k + 1)!}$

In [4]:
def ex_1(x, n):
    termeven, termodd, sumeven, sumodd = 1, -x, 1, -x
    k = 2
    while k < n:
        termeven *= (x**2)/(2*k*(k - 1))
        termodd *= (x**2)/(2*k*((2*k) + 1))
        sumeven += termeven
        sumodd -= termodd
        k += 1
        return sumeven + sumodd

Another method is to loop over `j` from 1 to $n$, let `term = term * x/j` and add the term to the initial `sum` that was set to 0. But which method is better? Mathematically, both are good but computationally?

In [5]:
def ex_2(x, n):
    j, tot_sum, term = 1, 1, 1
    while j < n:
        term *= x/j
        sign = (-1)**j
        tot_sum += term * sign
        j += 1
        return tot_sum

print(ex_2(3, 500), np.exp(-3))

-2.0 0.049787068367863944


In [6]:
yn = 'Y'
while yn == 'Y' or 'y':
    x = int(input('Enter the value of x: '))
    n = int(input('Enter the value of n: '))
    print('Sum 1: ', ex_1(x, n), 'Sum 2: ', ex_2(x, n), 'Actual Value: ', np.exp(-x))
    yn = input('Do you want to continue? [Y/N] ')
    if yn != 'Y':
        break

Enter the value of x:  1
Enter the value of n:  7


Sum 1:  0.30000000000000004 Sum 2:  0.0 Actual Value:  0.36787944117144233


Do you want to continue? [Y/N]  Y
Enter the value of x:  1
Enter the value of n:  50


Sum 1:  0.30000000000000004 Sum 2:  0.0 Actual Value:  0.36787944117144233


Do you want to continue? [Y/N]  N


The first method is computationally more efficient. Some key takeaways from this:

1. Do not subtract large numbers, especially if the difference is expected to be small
2. Isolate positive and negative terms for oscillatory series

### Recursive *vs* Dynamic Programming

Let us look back at the golden mean. In recursive programming, we did something along the lines of

`def gmean(n)`

...

`return gmean(n - 2) + gmean(n - 1)`

On the other hand, dynamic programming was applied along the lines of

`def gmean_dyn(n)`

`gld_num = gmean_dyn(n - 2) + gmean_dyn(n - 1)`
    
`arr[n] = gld_num`
    
`return gld_num`
    
`arr = {}`

`print(gmean_dyn(n))`

Dynamic programming could also involve something called a *tree structure*. A dynamic programme algorithm solves a complex problem by breaking it down into its smaller parts (nodes) and stores the results to these subproblems after computing them once. 

## Figuring Out Using Transfer Matrices (Problem 8 Cont.)

Now, we can write a 'smarter' programme for the golden mean. We know that $\phi^n = \phi^{n-1} + \phi^{n-2}$. Let us take $\begin{pmatrix} \phi^n \\ \phi^{n-1} \end{pmatrix}$, considering it as a two-dimensional vector. Now, we can write the original recurrence relation in matrix form as

$
\begin{pmatrix} \phi^n \\ \phi^{n-1} \end{pmatrix} =
\begin{pmatrix} 1 & 1 \\ 1 & 0 \end{pmatrix} \times
\begin{pmatrix} \phi^{n-1} \\ \phi^{n-2} \end{pmatrix}
$,

where $T = \begin{pmatrix} 1 & 1 \\ 1 & 0 \end{pmatrix}$ is something called the *transfer matrix* (in fact, this was also used to solve the Ising model!) To approximate the golden ratio to some *n* power, we just essentially repeat this process.

$
\implies \begin{pmatrix} \phi^n \\ \phi^{n-1} \end{pmatrix} = T^{(n-1)} \times
\begin{pmatrix} \phi^1 \\ \phi^0 \end{pmatrix}
$

where

$\begin{pmatrix} \phi^1 \\ \phi^0 \end{pmatrix} = \begin{pmatrix} \phi \\ 1 \end{pmatrix}$

It is important to diagonalise this so that we can use the basis for the next calculation. This is a very efficient method to compute in O$(\ln{n})$ time instead of O$(n)$ or O$(2^{n})$.

In [47]:
def expo(T, n):
    result = np.identity(2) # starting with an identity matrix
    base = T
    while n > 0:
        if n % 2 == 1:
            result = np.dot(result, base)
        base = np.dot(base, base)
        n //= 2
    return result

# this is a function for fast binary power calculation that computes in logarithmic time. We are reducing n step-by-step in binary

In [48]:
phi = (1 + np.sqrt(5))/2

def transfer_matrix(n):
    if n == 0:
        return 1
    elif n == 1:
        return phi
        
    T = np.array([[1, 1], [1, 0]], dtype = np.float64)
    T_n = expo(T, n - 1)

    # basis
    phi_0, phi_1 = 1, phi

    result = np.dot(T_n, np.array([phi_0, phi_1]))
    return result[0]

In [51]:
N = int(input('Number of steps (n): '))
print(transfer_matrix(N))

Number of steps (n):  10


110.01315561749642


In [56]:
def stab(T):
    eigenval, x = np.linalg.eig(T)
    print("Eigenvalues: ", eigenval)

    # checking stability
    stable = all(abs(l) < 1 for l in eigenval)
    if stable:
        print('Stable (lambda < 1)')
    else:
        print('Unstable (some lambda >= 1)')

stab(T)

Eigenvalues:  [ 1.61803399 -0.61803399]
Unstable (some lambda >= 1)


The eigenvalues here are $(1 + \sqrt{5})/2$ and $(1 - \sqrt{5})/2$